In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/jobs-on-naukricom/naukri_com-job_sample.csv


In [136]:
df = pd.read_csv("../input/jobs-on-naukricom/naukri_com-job_sample.csv")

In [137]:
df.shape

(22000, 14)

In [138]:
df.head(10)

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,210516002263,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8
1,find live infotech,UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...,0 - 0 yrs,Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,210516002391,Chennai,Work Based Onhome Based Part Time.,60.0,"1,50,000 - 2,50,000 P.A. 20000",2016-05-21 19:30:00 +0000,NaN,Marketing,d4c72325e57f89f364812b5ed5a795f0
2,Softtech Career Infosystem Pvt. Ltd,UG: Any Graduate - Any Specialization PG:Any P...,4 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,101016900534,Bengaluru,Pl/sql Developer - SQL,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,c47df6f4cfdf5b46f1fd713ba61b9eba
3,Onboard HRServices LLP,UG: Any Graduate - Any Specialization PG:CA Do...,11 - 15 yrs,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,81016900536,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,Accounts,115d28f140f694dd1cc61c53d03c66ae
4,Spire Technologies and Solutions Pvt. Ltd.,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,6 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,120916002122,Bengaluru,JAVA Technical Lead (6-8 yrs) -,4.0,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,a12553fc03bc7bcced8b1bb8963f97b4
5,PFS Web Global Services Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:MCA - ...,2 - 5 yrs,IT-Software / Software Services,Job Description Send me Jobs like this We ar...,131016005070,Bengaluru,WALK IN - As400 Developer - Pfsweb Global Serv...,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,8c3af9062ea835b0965779e2c7faac76
6,Kinesis Management Consultant Pvt. Ltd,NaN,1 - 3 yrs,IT-Software / Software Services,Job Description Send me Jobs like this exper...,131016004451,"Delhi NCR, Mumbai, Bengaluru, Kochi, Greater N...",PHP Developer,2.0,"3,00,000 - 6,50,000 P.A",2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,98f84958cd6409386e7f0c9e447b8510
7,Agile HR consultancy Pvt. Ltd. hiring for Ross...,"UG: Diploma - Any Specialization, Electrical, ...",2 - 7 yrs,Aviation / Aerospace Firms,Job Description Send me Jobs like this Job D...,121016002995,Bengaluru,Member Technical Staff-wire Harness/cable Harn...,20.0,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,Production,c811c10568fdbfdb3d5e633c99c914b4
8,HANSUM INDIA ELECTRONICS PVT.LTD.,"UG: Diploma - Any Specialization, Electronics/...",1 - 3 yrs,Industrial Products / Heavy Machinery,Job Description Send me Jobs like this Indep...,131016002120,Bengaluru,Team Leader,2.0,"1,00,000 - 2,00,000 P.A",2016-10-13 16:20:55 +0000,NaN,Production,a40cb270401af0c246dbe60dcf219262
9,Accenture,UG: Any Graduate - Any Specialization PG:Any P...,1 - 5 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Overa...,121016901354,Bengaluru,German Translator,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,ITES,7774df1c672c0b92486da8b36a721638


In [139]:
null_values = df.isnull().sum()
print(null_values)

company                    4
education               1996
experience                 4
industry                   5
jobdescription             4
jobid                      0
joblocation_address      501
jobtitle                   0
numberofpositions      17536
payrate                   97
postdate                  23
site_name              18013
skills                   528
uniq_id                    0
dtype: int64


* There are many missing values in site_name and numberofpositions , it would be better to drop these columns, site_name is in fact a non-numerical data therefore imputation is not possible

In [140]:
df.drop(["numberofpositions","site_name"],axis=1,inplace=True)

In [141]:
df.isnull().sum()

company                   4
education              1996
experience                4
industry                  5
jobdescription            4
jobid                     0
joblocation_address     501
jobtitle                  0
payrate                  97
postdate                 23
skills                  528
uniq_id                   0
dtype: int64

In [142]:
df.joblocation_address.unique()

array(['Chennai', 'Bengaluru',
       'Mumbai, Bengaluru, Kolkata, Chennai, Coimbatore, Tamilnadu', ...,
       'Mumbai, Bengaluru, Hyderabad, Kolkata',
       'Mumbai, Chennai, Delhi, Bengaluru, Hyderabad, Pune',
       'Bengaluru, Malaysia'], dtype=object)

In [143]:
#we can replace missing values in with 5 most frequent job_locations
#in some cases job location is shown in job title, we should check this first
df['joblocation_address'].value_counts()

Bengaluru/Bangalore                                                                                              3775
Mumbai                                                                                                           2348
Bengaluru                                                                                                        1453
Hyderabad / Secunderabad                                                                                         1246
Chennai                                                                                                          1204
                                                                                                                 ... 
Chennai , Saudi Arabia                                                                                              1
Hyderabad / Secunderabad , Delhi/NCR(National Capital Region) , Gurgaon                                             1
Bengaluru/Bangalore , Chennai , Ernakulam / Kochi/ Cochi

In [144]:
index_of_missing_jobl = df[df['joblocation_address'].isnull()].index.tolist()

In [145]:
frequent_locs=["Bengaluru/Bangalore","Mumbai","Hyderabad / Secunderabad","Chennai"]
for index in index_of_missing_jobl:
    if df.loc[index].jobtitle.split()[-1] in df['joblocation_address'].unique():
        df.at[index,"joblocation_address"]=df.loc[index].jobtitle.split()[-1]
    else:
        df.at[index,"joblocation_address"] =np.random.choice(frequent_locs)
        
    

In case of replacing missing values in "education", from real life scenario there could be a strong correlation between "jobtitle" and "education"


In [146]:
columns_to_be_dropped = ["company","education","industry","jobdescription","payrate","postdate","skills"]
index=[]
for column in columns_to_be_dropped:
    index += df[df[column].isnull()].index.tolist()
    

unique_index=set(index)
rows_to_be_removed = list(unique_index)


    

In [147]:
df.drop(df.index[rows_to_be_removed],inplace=True)

In [133]:
df.isnull().sum()

company                  0
education                0
experience               0
industry                 0
jobdescription           0
jobid                    0
joblocation_address      0
jobtitle                 0
payrate                  0
postdate                 0
skills                 458
uniq_id                  0
dtype: int64

In [ ]:
df.isnull().sum()